In [ ]:
import parsy

In [ ]:
auto_pick_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.999.ifrac"
"""

In [ ]:
auto_pick_output_text

In [ ]:
newline = parsy.string('\n')
get_observations = parsy.string("INFO:root:len(observation_set.GetObservations())=") >> parsy.regex(r'\d+').map(int)
output_path_name = (parsy.string('INFO:root:Wrote changes to') >>
                    parsy.regex(r' "c:\\src\\Orchid.IntegrationTestData\\frankNstein_Bakken_UTM13_FEET.\d{3}.ifrac"'))
    
@parsy.generate
def auto_pick_output():
    yield parsy.string("INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'")
    yield newline >> parsy.string("INFO:root:len(native_project.ObservationSets.Items)=2")
    yield newline >> parsy.string("INFO:root:observation_set.Name='ParentWellObservations'")
    yield newline >> get_observations
    yield newline >> parsy.string("INFO:root:observation_set.Name='Auto-picked Observation Set3'")
    get_observations_count = yield newline >> get_observations
    yield newline >> output_path_name
    yield newline
    
    return get_observations_count

In [ ]:
auto_pick_output.parse(auto_pick_output_text)